In [58]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

# Django ORM CRUD Operations Tutorial

CRUD stands for Create, Read, Update, and Delete - the four basic operations for database management systems. In this tutorial, we'll explore how to perform these operations using Django ORM with the NewsData model.

## NewsData Model Structure

Let's first examine the structure of our NewsData model to understand its fields.

In [10]:
# Inspect the NewsData model fields
print('NewsData Model Fields:')
for field in NewsData._meta.get_fields():
    print(f'- {field.name}: {field.get_internal_type()}')

# Display total count of news entries
print(f'\nTotal news entries in database: {NewsData.objects.count()}')

NewsData Model Fields:
- item_id: CharField
- date: DateField
- category: CharField
- title: TextField
- content: TextField
- sentiment: FloatField
- top_key_freq: TextField
- tokens_v2: TextField
- entities: TextField
- token_pos: TextField
- link: CharField
- photo_link: CharField

Total news entries in database: 213


## 1. Create (C) - Adding New Data

The 'Create' operation allows us to insert new records into the database. Here are different ways to create new NewsData objects.

In [ ]:
# Method 1: Create using the constructor and save() method
new_news1 = NewsData(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news1",
    category="測試類別",
    content="This is a sample content about technology news for demonstration.",
    date=datetime.now().date(),
    photo_link="https://example.com/image1.jpg"
)
# Uncomment to save: new_news1.save()

# Method 2: Using objects.create() method
# This creates and saves in one step
new_news2 = NewsData.objects.create(
    title="馬斯克蒞臨高科大",
    link="https://example.com/news2",
    category="測試類別",
    content="This is sample content about international news for demonstration.",
    date=datetime.now().date(),
    photo_link=None  # Shows that photo_link can be optional
)

# Method 3: Bulk creation (for adding multiple records at once)
news_list = [
    NewsData(
        title=f"Bulk Example {i}", 
        link=f"https://example.com/bulk{i}",
        category="測試類別",
        content=f"Bulk created content {i}",
        date=datetime.now().date()
    ) for i in range(3, 6)
]
# Uncomment to save: NewsData.objects.bulk_create(news_list)

print(f"Created news item: {new_news2.title} (ID: {new_news2.item_id})")
print("Note: The other examples are commented out to prevent adding test data to your database.")

Created news item: Example News Title 2 (ID: )
Note: The other examples are commented out to prevent adding test data to your database.


In [21]:

# Clean up - delete the example we created
new_news2.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 2. Read (R) - Querying Data

The 'Read' operation retrieves data from the database. Django ORM provides powerful ways to query data.

In [22]:
# Basic querying examples

# 1. Get all records
all_news = NewsData.objects.all()
print(f"Total news items: {all_news.count()}")


Total news items: 213


In [23]:

# 2. Get a specific record by ID
try:
    first_id = NewsData.objects.first().item_id
    specific_news = NewsData.objects.get(item_id=first_id)
    print(f"\nNews by ID {first_id}: {specific_news.title}")
except:
    print("Couldn't retrieve news by ID")



News by ID acn_20220313_20: 劉兆佳：香港疫情打擊威信 北京將加強命令特首


In [24]:

# 3. Filter records
# Get news from a specific category
tech_news = NewsData.objects.filter(category="科技")
print(f"\nTechnology news count: {tech_news.count()}")
if tech_news.exists():
    print(f"Example tech news: {tech_news.first().title}")



Technology news count: 20
Example tech news: 在台推動城市4.0 西門子：智慧城市下一步演變


In [27]:

# 4. Complex filtering
# Get news from the last 7 days
one_week_ago = datetime.now().date() - timedelta(days=7)
recent_news = NewsData.objects.filter(date__gte=one_week_ago)
print(f"\nNews from the last 7 days: {recent_news.count()}")



News from the last 7 days: 0


In [40]:

# 5. Ordering
latest_news = NewsData.objects.order_by('-date')[:5]  # Get 5 most recent news
print("\n5 most recent news:")
for news in latest_news:
    print(f"- {news.item_id} {news.date}:{news.category} {news.title}")



5 most recent news:
-  2025-05-05:刪除測試 Delete Demo 1
- aipl_20220314_1 2022-03-14:政治 外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾
- aipl_20220314_2 2022-03-14:政治 幻象飛官獲救 空軍：掌握跳傘時機做出好示範
- aipl_20220314_3 2022-03-14:政治 法製幻象零附件取得不易 空軍：妥善率沒問題
- aipl_20220314_4 2022-03-14:政治 游錫堃：無禁止反質詢規定  擇期協商討論入法可能


In [29]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title')[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- ('外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾',)
- ('幻象飛官獲救 空軍：掌握跳傘時機做出好示範',)
- ('法製幻象零附件取得不易 空軍：妥善率沒問題',)


In [30]:
titles_only

<QuerySet [('外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾',), ('幻象飛官獲救 空軍：掌握跳傘時機做出好示範',), ('法製幻象零附件取得不易 空軍：妥善率沒問題',)]>

In [28]:

# 6. Specific fields selection
titles_only = NewsData.objects.values_list('title', flat=True)[:3]
print("\nSome news titles:")
for title in titles_only:
    print(f"- {title}")



Some news titles:
- 外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾
- 幻象飛官獲救 空軍：掌握跳傘時機做出好示範
- 法製幻象零附件取得不易 空軍：妥善率沒問題


In [ ]:

# 7. Text search (simple contains)
keyword = "台灣"
matching_news = NewsData.objects.filter(content__contains=keyword).count()
print(f"\nNews mentioning '{keyword}': {matching_news}")

In [3]:
from django.db.models import Q, Max, F
from collections import Counter

## 3. Update (U) - Modifying Data

The 'Update' operation modifies existing records in the database. There are several ways to update data in Django.

In [31]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [32]:

# Method 1: Instance update
print(f"Original title: {test_news.title}")
test_news.title = "Updated Test News Title"
test_news.category = "更新測試"
test_news.save()
print(f"Updated title: {test_news.title}, Category: {test_news.category}")


Original title: Test News for Update Demo
Updated title: Updated Test News Title, Category: 更新測試


In [33]:

# Method 2: QuerySet update (more efficient for multiple records)
# This updates all matching records without loading them into memory
update_count = NewsData.objects.filter(item_id=test_news.item_id).update(
    content="This content was updated using QuerySet.update() method.",
    photo_link="https://example.com/updated-image.jpg"
)
print(f"\nRecords updated via QuerySet: {update_count}")



Records updated via QuerySet: 1


In [35]:
test_news.delete()

(1, {'app_user_keyword_db.NewsData': 1})

## 4. Delete (D) - Removing Data

The 'Delete' operation removes records from the database. Django provides various methods to delete data.

In [52]:
# Let's demonstrate updates with a temporary test record
test_news = NewsData.objects.create(
    item_id="item-123",  # This will be auto-generated by the database
    title="Test News for Update Demo",
    link="https://example.com/test-update",
    category="刪除測試",
    content="This is a temporary record to show update operations.",
    date=datetime.now().date()
)


In [42]:
NewsData.objects.get(item_id="item-123").delete()

(1, {'app_user_keyword_db.NewsData': 1})

In [53]:
NewsData.objects.filter( category="刪除測試")

<QuerySet [<NewsData: 2025-05-05: Test News for Update Demo>]>

In [54]:
NewsData.objects.filter( category="刪除測試").delete()

(1, {'app_user_keyword_db.NewsData': 1})

# filter_database_fullText

In [ ]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        # q_objects會長這樣： Q(content__contains=kw1) | Q(content__contains=kw2) | Q(content__contains=kw3)
        # 這樣就可以用在filter裡面了
        queryset = queryset.filter(q_objects)
    
    return queryset

In [5]:

user_keywords = ['烏克蘭', '俄羅斯']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [6]:
for news in queryset:
    print(news.title)

俄侵烏克蘭全球憤慨 臉書放寬規定讓人抒發怒火
Apple新品一次看 iPhone SE售1萬3900元起iPad Air有5色[影]
北京冬季帕運閉幕 帕委會主席談和平與希望
俄烏談判露曙光 油價下滑5%
華碩停止對俄羅斯出貨 捐款3000萬賑濟烏克蘭
油價若續漲 朱澤民：今年CPI有可能超過2%
澳洲擴大制裁俄羅斯 歐盟要凍結切爾西老闆資產
俄羅斯提核協議新要求 伊朗外長將赴莫斯科討論
2022酷寒演習展開 3萬北約兵力集結挪威
俄國遭制裁降價求售石油和商品 印度考慮採購
烏克蘭戰事中國疫情添不安 亞股多數收黑
路透社：美中高層已在羅馬會晤
借鑑烏克蘭核電廠遭攻 日研議核廠設專屬警備隊
烏俄進行第4輪談判 烏克蘭代表稱雙方溝通困難
國際博物館協會發聲拒絕戰爭 吳思瑤籲故宮跟進
戰爭時文物如何疏散  故宮3個月內擬對策7月推演
香港恆指暴跌千點 失守2萬點創6年新低


In [7]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [8]:
news_items

<QuerySet [{'category': '科技', 'title': '俄侵烏克蘭全球憤慨 臉書放寬規定讓人抒發怒火', 'link': 'https://www.cna.com.tw/news/ait/202203110088.aspx', 'photo_link': 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220311/2000x1391_0522240424184.jpg'}, {'category': '科技', 'title': 'Apple新品一次看 iPhone SE售1萬3900元起iPad Air有5色[影]', 'link': 'https://www.cna.com.tw/news/ait/202203090006.aspx', 'photo_link': 'https://imgcdn.cna.com.tw/www/webphotos/WebCover/420/20220309/800x600_644221551345.jpg'}, {'category': '運動', 'title': '北京冬季帕運閉幕 帕委會主席談和平與希望', 'link': 'https://www.cna.com.tw/news/aspt/202203130215.aspx', 'photo_link': None}]>

In [55]:
list(queryset.values_list('date', flat=True)) # flat=True to get a flat list 否則會是 tuple

[datetime.date(2022, 3, 11),
 datetime.date(2022, 3, 9),
 datetime.date(2022, 3, 13),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14),
 datetime.date(2022, 3, 14)]

In [56]:
latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']

In [57]:
latest_date

datetime.date(2022, 3, 14)

In [ ]:
from django.db.models import Avg, Count, Min, Max, Sum
from django.db.models.functions import Extract

# Aggregate是用來計算總和、平均值、最小值、最大值等統計數據的函數
# 這些函數可以用來對查詢集進行聚合計算，並返回一個字典，其中包含計算結果的鍵值對
# Basic aggregation examples
aggregations = NewsData.objects.aggregate(
    count=Count('item_id'),
    oldest=Min('date'),
    newest=Max('date'),
)
print("Basic aggregations:")
for key, value in aggregations.items():
    print(f"- {key}: {value}")


Basic aggregations:
- count: 213
- oldest: 2022-03-03
- newest: 2022-03-14


In [61]:

# Category-based aggregation
category_counts = NewsData.objects.values('category').annotate(
    count=Count('item_id')
).order_by('-count')[:5]  # Top 5 categories
print("\nTop 5 categories by count:")
for item in category_counts:
    print(f"- {item['category']}: {item['count']} articles")


Top 5 categories by count:
- 運動: 20 articles
- 證卷: 20 articles
- 科技: 20 articles
- 社會: 20 articles
- 產經: 20 articles


In [ ]:
# annotate是用來在查詢集中添加計算字段的函數
# 這些計算字段可以用來進行進一步的過濾、排序或分組操作
# values是用來選擇查詢集中的特定字段的函數 字段就是資料庫中的欄位
# Date-based aggregation (articles per day)
date_counts = NewsData.objects.values('date').annotate(
    count=Count('item_id')
).order_by('-date')[:7]  # Last 7 days with data

print("\nArticles per day (most recent 7 days with data):")
for item in date_counts:
    print(f"- {item['date']}: {item['count']} articles")



Articles per day (most recent 7 days with data):
- 2022-03-14: 175 articles
- 2022-03-13: 14 articles
- 2022-03-12: 4 articles
- 2022-03-11: 4 articles
- 2022-03-10: 2 articles
- 2022-03-09: 8 articles
- 2022-03-08: 3 articles
